In [52]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')
from utils.binary import get_blackhole_binary
from utils.star import make_mesa_star
from amuse.plot import *
from amuse.community.ph4 import Ph4
from amuse.community.fi import Fi
from amuse.couple.bridge import Bridge
from amuse.units import units, constants, nbody_system
from amuse.ext.sink import SinkParticles

In [25]:
binary = get_blackhole_binary(100,100)
star = make_mesa_star()

offset = [1, 0, 0] | units.m                              # <-- increase for smoother/lower-cost interaction
vrel   = [-200, 0, 0] | units.km/units.s  
star.position += offset
star.velocity += vrel
gas, core = star[star.name == "gas"],  star[star.name == "core"]

In [56]:

converter_grav = nbody_system.nbody_to_si(1 | units.AU, 1 | units.MSun)
gravity = Ph4(converter_grav)
gravity.parameters.epsilon_squared = (0.01 | units.AU)**2     # <-- increase to 0.05–0.1 AU for faster/softer forces
gravity.parameters.timestep_parameter = 0.03                   # <-- increase to 0.1 for faster runs

gravity.particles.add_particles(binary)
gravity.particles.add_particle(core)


In [57]:

sink_radius = 1.0 | units.AU                                  # <-- decrease to 0.2 AU for faster, smoother runs
sinks = SinkParticles(gravity, sink_radius)
sinks.sink_particles.add_particles(binary)

bh_accreted_mass = 0 | units.MSun
initial_envelope_mass = gas.total_mass()

AttributeError: key

In [51]:
converter_hydro = nbody_system.nbody_to_si(1 | units.RSun, 1 | units.MSun)
hydro = Fi(converter_hydro)
hydro.gas_particles.add_particles(gas)
hydro.parameters.use_hydro_flag = True
hydro.parameters.self_gravity_flag = True                     # <-- set False for BIG speed boost
hydro.parameters.gamma = 5.0/3.0
hydro.parameters.timestep = 5e3 | units.s                     # <-- increase to 2e4 for faster runs
hydro.parameters.artificial_viscosity_alpha = 0.5             # <-- increase to 1.0 to stabilize when fast-mode

hydro.evolve_model(0.2 | units.day)                           # <-- reduce to 0.05 day to speed up “relaxation”


[]

In [ ]:

bridge = Bridge(use_threading=False)
bridge.add_system(gravity, (hydro,))
bridge.add_system(hydro, (gravity,))
bridge.timestep = 0.2 | units.day                              # <-- increase to 1 day for speed (less accuracy)